In [8]:
import pickle
import imageio
import sys
import pdb
import os
import cv2
import numpy as np
import trimesh

pickle_name = '04c195f4e482e1e1ad06327d014f2fdd'
OBJNAME = 'person1'


os.makedirs(f'output/{OBJNAME}/LASR/FlowFW/Full-Resolution/{OBJNAME}',exist_ok=True)
os.makedirs(f'output/{OBJNAME}/LASR/FlowBW/Full-Resolution/{OBJNAME}',exist_ok=True)
os.makedirs(f'output/{OBJNAME}/LASR/Annotations/Full-Resolution/{OBJNAME}',exist_ok=True)
os.makedirs(f'output/{OBJNAME}/LASR/JPEGImages/Full-Resolution/{OBJNAME}',exist_ok=True)
os.makedirs(f'output/{OBJNAME}/LASR/Meshes/Full-Resolution/{OBJNAME}',exist_ok=True)
# os.makedirs(f'output/{OBJNAME}/LASR/FlowFW/Full-Resolution/r{OBJNAME}',exist_ok=True)
# os.makedirs(f'output/{OBJNAME}/LASR/FlowBW/Full-Resolution/r{OBJNAME}',exist_ok=True)


with open(f'./MeshSeqs/{pickle_name}.pkl', 'rb') as file:
    frames_dict = pickle.load(file)

for i in range(len(frames_dict)):
    imageio.imsave(f'output/{OBJNAME}/LASR/JPEGImages/Full-Resolution/{OBJNAME}/{i:05d}.png',frames_dict[i]['rgb_frame'])
    # miss mask

    mesh = trimesh.Trimesh(vertices=frames_dict[i]['mesh_coords_3d'][0,...])
    trimesh.exchange.export.export_mesh(mesh,f'output/{OBJNAME}/LASR/Meshes/Full-Resolution/{OBJNAME}/{i:05d}.obj')

# save gif
imageio.mimsave(f'output/{OBJNAME}/{OBJNAME}.gif',[frames_dict[i]['rgb_frame'] for i in range(len(frames_dict))])



fw = np.array([frames_dict[i]['forward_optical_flow'] for i in range(len(frames_dict))])[:-1,...]
bw = np.array([frames_dict[i]['backward_optical_flow'] for i in range(len(frames_dict))])[1:,...]


# write optical flow and occlusion map in LASR format
def write_pfm(path, image, scale=1):
    """Write pfm file.

    Args:
        path (str): pathto file
        image (array): data
        scale (int, optional): Scale. Defaults to 1.
    """

    with open(path, "wb") as file:
        color = None

        if image.dtype.name != "float32":
            raise Exception("Image dtype must be float32.")

        image = np.flipud(image)

        if len(image.shape) == 3 and image.shape[2] == 3:  # color image
            color = True
        elif (
            len(image.shape) == 2 or len(image.shape) == 3 and image.shape[2] == 1
        ):  # greyscale
            color = False
        else:
            raise Exception("Image must have H x W x 3, H x W x 1 or H x W dimensions.")

        file.write("PF\n".encode() if color else "Pf\n".encode())
        file.write("%d %d\n".encode() % (image.shape[1], image.shape[0]))

        endian = image.dtype.byteorder

        if endian == "<" or endian == "=" and sys.byteorder == "little":
            scale = -scale

        file.write("%f\n".encode() % scale)

        image.tofile(file)

for i in range(len(fw)):
    f = fw[i,...]
    ones = np.ones_like(f[...,:1])  
    f = np.concatenate([f[...,1:], f[...,:1], ones],-1)
    b = np.concatenate([-bw[i,...,1:],-bw[i,...,:1], ones],-1)

    f = np.flip(f,0)
    b = np.flip(b,0)
    
    write_pfm(f'output/{OBJNAME}/LASR/FlowFW/Full-Resolution/{OBJNAME}/flo-{i:05d}.pfm',f)
    write_pfm(f'output/{OBJNAME}/LASR/FlowBW/Full-Resolution/{OBJNAME}/flo-{i+1:05d}.pfm',b)
    write_pfm(f'output/{OBJNAME}/LASR/FlowFW/Full-Resolution/{OBJNAME}/occ-{i:05d}.pfm',np.ones_like(f))
    write_pfm(f'output/{OBJNAME}/LASR/FlowBW/Full-Resolution/{OBJNAME}/occ-{i+1:05d}.pfm',np.ones_like(b))


In [9]:
cv2.imread('output/airplane/LASR/JPEGImages/Full-Resolution/airplane/00000.png')

array([[[162, 162, 162],
        [162, 162, 162],
        [162, 162, 162],
        ...,
        [162, 162, 162],
        [162, 162, 162],
        [162, 162, 162]],

       [[162, 162, 162],
        [163, 163, 163],
        [162, 162, 162],
        ...,
        [162, 162, 162],
        [162, 162, 162],
        [161, 161, 161]],

       [[162, 162, 162],
        [162, 162, 162],
        [161, 161, 161],
        ...,
        [163, 163, 163],
        [161, 161, 161],
        [163, 163, 163]],

       ...,

       [[162, 162, 162],
        [162, 162, 162],
        [162, 162, 162],
        ...,
        [162, 162, 162],
        [162, 162, 162],
        [161, 161, 161]],

       [[162, 162, 162],
        [162, 162, 162],
        [162, 162, 162],
        ...,
        [162, 162, 162],
        [161, 161, 161],
        [162, 162, 162]],

       [[161, 161, 161],
        [162, 162, 162],
        [161, 161, 161],
        ...,
        [162, 162, 162],
        [162, 162, 162],
        [161, 161, 161]]